In [253]:
%cd /content/drive/My Drive/internshala

/content/drive/My Drive/internshala


In [0]:
import cv2
import os
import numpy as np
import pandas as pd
import cv2
import sys
import json
import re
import requests
import io
import tensorflow as tf
from imutils.object_detection import non_max_suppression
from matplotlib import pyplot as plt
from keras import backend as K
from google.colab.patches import cv2_imshow

In [0]:
#loading all images 
imgs = []
path = "/content/drive/My Drive/internshala"
valid_images = [".jpg"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    imgs.append(cv2.imread(os.path.join(path,f)))

In [0]:
reg = []
name = []
vno = []
eno = []
regdate = []
mdate = []
df = pd.DataFrame(columns = ('REGISTRATION_NO','NAME','VIN number', 'Engine number','Registration date','Mfg. date'))
#image1 = cv2.imread(imgs[0])
alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)
for i in range(0,len(imgs)):
  image2 = cv2.cvtColor(imgs[i], cv2.COLOR_BGR2GRAY)
  image = cv2.convertScaleAbs(image2, alpha=alpha, beta=beta)
  kernel = np.ones((2,2),np.float32)/9
  dst = cv2.filter2D(image,-1,kernel)
  # ocr.space is used as ocr api 
  _, compressedimage = cv2.imencode(".jpg",dst)
  file_bytes =io.BytesIO(compressedimage)
  url_api = "https://api.ocr.space/parse/image"
  result = requests.post(url_api,files={"first.jpg": file_bytes},data={"apikey":"c71dbd0b2b88957"})
  p = result.content.decode()
  p = json.loads(p)
  t = p.get("ParsedResults")[0].get("ParsedText")
  text_detected = t.upper().replace(" ","")
  text_detected = text_detected.replace("\r"," ").replace("\n","").replace("*","  ").replace("-","").replace(":","  ")
  #extracting reg_numbers.:
  try:
    mydog = re.compile("\s[A-Z0-9]{1,15}\d\s")
    r = mydog.findall(text_detected)[0]
    r = r.replace("NO:","")
    reg.append(r)
  except:
    reg.append("na")
    pass
# extrcting names
  try:
    mydog = re.compile("\sMR.[A-Z]+\s")
    r = mydog.findall(text_detected)[0]
    r =r.replace("MR.","")
    name.append(r)
  except:
    name.append("na")
    pass

# extracting vin_numbers
  try:
    mydog = re.compile("\s[A-Z0-9]{16}\d\s")
    r = mydog.findall(text_detected)[0]
    vno.append(r)
  except:
    vno.append("na")
    pass
# extracting Mfg date
  try:
    mydog = re.compile("\s\d+/\d{4}\s")
    r = mydog.findall(text_detected)[0]
    mdate.append(r)
  except:
    mdate.append("na")
    pass
# extracting Registration date
  try:
    mydog = re.compile("\s\d+/\d+/\d{4}\s")
    r = mydog.findall(text_detected)[0]
    regdate.append(r)
  except:
    regdate.append("na")
    pass
# extracting engine number date
  try:
    mydog = re.compile("\s[A-Z0-9]{11}\d\s")
    r = mydog.findall(text_detected)[0]
    eno.append(r)
  except:
    eno.append("na")
    pass

In [0]:
df['REGISTRATION_NO'] = reg
df['NAME'] = name
df ['VIN number'] = vno
df['Engine number'] = eno
df['Registration date'] = regdate
df['Mfg. date'] = mdate

In [258]:
df

,REGISTRATION_NO,NAME,VIN number,Engine number,Registration date,Mfg. date
0,na,na,na,na,30/12/2024,na
1,HRIOP5470,na,MALCM41VR9M079263,na,31/12/2009,na
2,34984,na,na,na,11/01/2003,na
3,HRIOM5657,na,na,na,29/08/2008,na
4,80092011,na,na,na,28/08/2023,na
5,DL8CP1150,na,MA3EMD81S00167856,na,na,na
6,002425,na,na,na,20/07/2030,na
7,RZF14,na,na,na,na,na
8,DLICU8248,na,na,na,na,na
9,na,na,na,na,na,na


In [0]:
df.to_csv('result.csv')

In [260]:
t

"Namc/Brand of Vehicle\r\nMaker's Name\r\nVehicle Class\r\nNo.of Cylinders\r\nUnladen\r\nof Body\r\n11.1).AJ Lease Agg.\r\nDealer's Name\r\nAddress\r\nIQgistration Valid Upto\r\nRoad Tax Paid Upto\r\nDate of Issue\r\nACCENT-MT-GLE\r\nL,MV (MOTOR CAR)\r\n4 Cubic Capacity 1495CC\r\n1039K colour GTCWJIITE\r\nSEDAN\r\nSBOP\r\nMALWA AUTO SALES\r\nKUNI)LI\r\n19/01/2025\r\nOne Time\r\n03/03/2010\r\nSpecimen Signature\r\nof RegÅ Owner\r\n14.09 2011\r\n16 G\r\n"